In [4]:
import numpy as np
import math
import matplotlib.pyplot as plt  
from sklearn.metrics import mean_squared_error
import MySQLdb

from keras.layers import Dropout
from keras.layers import Dense,Input
from keras.models import Sequential,Model
from keras.callbacks import EarlyStopping
from sklearn.linear_model import LinearRegression

import sys
import xlwt
##get data##
import MySQLdb
from sklearn.preprocessing import MinMaxScaler


def transfromData(trainRaw, testRaw,windosSize):  ##Train ratial, train, test
    sc = MinMaxScaler(feature_range = (0, 1))
    #npRaw = np.array(rawData)
    #scaledData = sc.fit_transform(npRaw.reshape(-1,1))  
    #trainData = np.array(scaledData[:trainCount]).reshape(-1,1)##train 
    #testData = np.array(scaledData[-testCount:]).reshape(-1,1)
    
    npTrain = sc.fit_transform(np.array(trainRaw).reshape(-1,1))
    npTest = sc.fit_transform(np.array(testRaw).reshape(-1,1))
    

    X_train, y_train = splitXy(npTrain,windosSize)
    X_test, y_test = splitXy(npTest,windosSize)
    return sc, X_train, y_train, X_test, y_test

def splitXy(data,windosSize):
    windows = []
    for i in range(windosSize, data.shape[0]):
        windows.append(data[i-windosSize:i, 0])
    np.random.shuffle(windows)
    X = []
    y = []
    for i in range(len(windows)):
        X.append(windows[i][:windosSize-1])
        y.append(windows[i][-1:])
    X, y = np.array(X), np.array(y)
    #X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X,y
def buildModel():
    dnn_1_input = Input(shape=(6,),name='dnn_1_input')
    dnn_1 = Dense(128,activation="tanh", name='dnn_1')(dnn_1_input)
    dnn_2 = Dense(64,activation="tanh", name='dnn_2')(dnn_1)
    dnn_3 = Dense(32,activation="tanh", name='dnn_3')(dnn_2)
    dnn_4 = Dense(1, activation='tanh', name='dnn_4')(dnn_3)

    model = Model(dnn_1_input,dnn_4)
    model.summary()
    model.compile(loss='mse',
                  optimizer='adam')
    return model
def getStationList(cursor):
    stationList = []
    queryString = "SELECT distinct station from cleandata"
    cursor.execute(queryString)
    results = cursor.fetchall()
    for row in results :
        val = str(row).strip("',()")
        stationList.append(val)
    stationList.remove('富貴角')##only 106少數資料
    return stationList
def Visualize():
    predicted = sc.inverse_transform(regressor.predict(X_test))
    originY = sc.inverse_transform (y_test)
    print("MSE : ["+str(mean_squared_error(predicted, originY))+"]")
    # Visualising the results
    plt.plot(originY[:100], color = 'red', label = 'Real')  
    plt.plot(predicted[:100], color = 'blue', label = 'Predicted ') 
    plt.legend()
    plt.show()
def writeExcelHead(sheet1,epochs,station):
    sheet1.write(0,1,station)
    raw = 1
    for e in range(epochs):
        sheet1.write(raw,0,e+1)
        raw+=1
def fetchData(cursor,station,windosSize):
    queryString = "SELECT * from cleandata where station='"+station+"' and item = 'PM2.5' and date NOT like '2017%'"
    cursor.execute(queryString)
    results = cursor.fetchall()

    trainRawData = []
    for row in results:
        for i in range(3,27):
            trainRawData.append(float(row[i]))
            
    queryString = "SELECT * from cleandata where station='"+station+"' and item = 'PM2.5' and date like '2017%'"
    cursor.execute(queryString)
    results = cursor.fetchall()

    testRawData = []
    for row in results:
        for i in range(3,27):
            testRawData.append(float(row[i]))

    sc, X_train, y_train, X_test, y_test = transfromData(trainRawData,testRawData,windosSize)
    return sc, X_train, y_train, X_test, y_test
def trainLR(sc, X_train, y_train, X_test, y_test):
    reg = LinearRegression().fit(X_train, y_train)
    predicted_LR = sc.inverse_transform(reg.predict(X_test))
    ori_y_test = sc.inverse_transform(y_test)
    mse = mean_squared_error(predicted_LR, ori_y_test)
    print(mse)
    return mse

In [7]:
import datetime
db=MySQLdb.connect(host="localhost",user="root", passwd="swater0", db="airdb", charset="utf8")
cursor = db.cursor()

stationList = getStationList(cursor)
col=1
for station in stationList:
    print("training : " +station)
    mseList = []
    for windowSize in range(2,22):
        sc, X_train, y_train, X_test, y_test = fetchData(cursor,station,windowSize)
        mse = trainLR(sc, X_train, y_train, X_test, y_test)
        mseList.append(mse)
    book = xlwt.Workbook(encoding="utf-8")
    sheet1 = book.add_sheet("Sheet1")
    writeExcelHead(sheet1,20,station)
    row = 1
    for mse in mseList:
        sheet1.write(row,1,mse)
        row+=1
    book.save("excelFiles/LR/LRresult"+station+".xls")
db.close()


training : 二林
26.6945365793706
26.578807749548137
26.194153428139987
24.626376347412233
24.694662715416502
24.678360903728382
24.437007824582565
24.456749915751335
24.438007209087676
24.338881599650872
24.32426753972409
24.30745168900124
24.33957166420531
24.331691969864316
24.31486990498477
24.344781169148753
24.337506732125984
24.33217886641303
24.279642488784493
24.27252822792625
training : 南投
18.568138707562618
16.642825331205263
15.794805259872595
15.719275911749689
15.562721761861145
15.57455469260841
15.554091057023507
15.55543046995516
15.539236801596031
15.523082427264484
15.5259088417534
15.526381697936843
15.520014105627117
15.52872341480947
15.513578309273722
15.502456751026633
15.493244207497256
15.471546074351018
15.47678120775217
15.483859997146858
training : 埔里
16.760297812627215
14.48949472595254
13.39041175928403
13.189419576031897
12.85288168062998
12.842479261260285
12.760543606900642
12.76366265797449
12.725270575054422
12.727620334984675
12.684818411846791
12.6486

13.531090087452194
13.54538249064867
13.53696154761878
13.537952423433568
13.528855148107455
13.52982023823048
13.524536371011191
13.522735277845426
13.526852629367884
13.521812602998141
13.516958950365968
13.497482255389038
13.482702765888886
13.471469464416563
13.46365510622171
13.461676387033364
training : 新莊
12.920856911904728
11.751304291709843
11.452408711550161
11.449236316202628
11.430718935702489
11.432321474914511
11.430997750066235
11.443487700806482
11.441731121545732
11.440948266301621
11.442986095423826
11.44774363681314
11.448549623795007
11.4498995273621
11.448954172174055
11.463865171333488
11.463974488650122
11.469240319958118
11.466740177303908
11.46867884689688
training : 松山
26.645102390012255
26.73312934461166
25.88988532002717
24.37928385218315
24.27569592728968
24.26759419710765
23.947952145078837
23.95355609729616
24.029071043188548
24.059320876189844
24.06412415328283
24.061189051805215
23.982837641607595
23.96733816175497
23.971395712488665
23.946716386152254


22.994321862736058
22.986468910281328
22.951736078675374
22.965483403327596
22.965816117906424
22.967234756051898
22.976672562592
22.985222676663508
22.986509425797554
22.955977444859588
22.965550064102743
22.91766639468984
training : 頭份
15.936092968217082
14.515975139894225
13.959300553940889
13.879655738959512
13.741174659284464
13.744828751087137
13.686232526005849
13.687037458849012
13.644144081291454
13.63937086353025
13.637002338548749
13.643380849549755
13.648150532457759
13.650074947180565
13.657327273274984
13.655754104972825
13.64757331044758
13.645355403134289
13.64537214953125
13.633545556201142
training : 臺東
5.66809053251483
5.620408001970862
4.899521306014556
4.87449432157883
4.773269041908817
4.7583981072486585
4.740040345417113
4.740112242440545
4.725279991421946
4.722484707762433
4.7114231155922495
4.7112343678362825
4.710583742082437
4.709568037709356
4.707206719777148
4.707308073704541
4.7067823019775075
4.705819983554837
4.7097631206087165
4.707643966629566
training

71.0880673296318
70.86998140032813
70.64742766072845
training : 大寮
22.772372833729435
21.029074300650752
19.751509752785594
19.724908191687426
19.660664982340297
19.662500031710547
19.576378381652646
19.56704836306531
19.545974258939893
19.497442476595925
19.395733804121242
19.39523554848382
19.401302663858498
19.343444666005993
19.37418362194895
19.375902958147194
19.35224029822326
19.29685700670872
19.21011587659078
19.17916370086657
training : 小港
42.21488417836975
43.08090565411394
43.59980381760447
46.895010363740255
47.23306224851673
47.2786499383042
48.24235777313024
48.128783550917184
48.09618856750107
48.15096711806309
48.01034153438062
47.95416660713435
47.89982643461308
47.76839625298828
47.838868296227595
47.72052911318453
47.46785181886597
47.53253553256321
47.34873420821716
47.204793439065526
training : 屏東
48.737729420445326
48.3465937876692
47.86254296931552
42.49369080340207
42.201339645712224
42.16026764899835
40.9245442136365
40.91593990602844
40.930539768867696
40.618